<a href="https://colab.research.google.com/github/GeeveGeorge/GFPGAN-for-Video-SR/blob/main/GFPGAN_for_Video_SR_by_Geeve_George.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Important Notes

* If you are using the free Colab tier , it works best for short length videos
* Just follow the instructions in the notebook and you should be fine!

*Colab Notebook prepared by Geeve George*

In [ ]:
%cd /content
!rm -rf GFPGAN
!git clone https://github.com/TencentARC/GFPGAN.git
%cd GFPGAN

# Set up the environment
# Install basicsr - https://github.com/xinntao/BasicSR
# We use BasicSR for both training and inference
!pip install basicsr
# Install facexlib - https://github.com/xinntao/facexlib
# We use face detection and face restoration helper in the facexlib package
!pip install facexlib
# Install other depencencies
!pip install -r requirements.txt
!python setup.py develop
!pip install realesrgan  # used for enhancing the background (non-face) regions
# Download the pre-trained model
!wget https://github.com/TencentARC/GFPGAN/releases/download/v0.2.0/GFPGANCleanv1-NoCE-C2.pth -P experiments/pretrained_models
# Now we use the V1.3 model for the demo
#!wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -P experiments/pretrained_models

/content
Cloning into 'GFPGAN'...
remote: Enumerating objects: 422, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 422 (delta 14), reused 27 (delta 10), pack-reused 381
Receiving objects: 100% (422/422), 5.36 MiB | 21.04 MiB/s, done.
Resolving deltas: 100% (207/207), done.
/content/GFPGAN
     |████████████████████████████████| 161 kB 5.3 MB/s 
     |████████████████████████████████| 5.8 MB 43.4 MB/s 
     |████████████████████████████████| 190 kB 48.1 MB/s 
  Created wheel for basicsr: filename=basicsr-1.3.5-py3-none-any.whl size=194483 sha256=48df6509f86742db143199c0f550327b1e07f18dd2ed6d260bb4f18020511abd
  Stored in directory: /root/.cache/pip/wheels/74/1b/d0/8659cf028233dd1e3bf282271009fbf037dfc4ab761f32a032
Successfully built basicsr
     |████████████████████████████████| 59 kB 605 kB/s 
     |████████████████████████████████| 177 kB 10.6 MB/s 
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.w

running develop
running egg_info
creating gfpgan.egg-info
writing gfpgan.egg-info/PKG-INFO
writing dependency_links to gfpgan.egg-info/dependency_links.txt
writing requirements to gfpgan.egg-info/requires.txt
writing top-level names to gfpgan.egg-info/top_level.txt
writing manifest file 'gfpgan.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
adding license file 'LICENSE'
writing manifest file 'gfpgan.egg-info/SOURCES.txt'
running build_ext
Creating /usr/local/lib/python3.7/dist-packages/gfpgan.egg-link (link to .)
Adding gfpgan 1.3.2 to easy-install.pth file

Installed /content/GFPGAN
Processing dependencies for gfpgan==1.3.2
Searching for yapf==0.32.0
Best match: yapf 0.32.0
Adding yapf 0.32.0 to easy-install.pth file
Installing yapf script to /usr/local/bin
Installing yapf-diff script to /usr/local/bin

Using /usr/local/lib/python3.7/dist-packages
Searching for tb-nightly==2.9.0a20220323
Best match: tb-nightly 2.9.0a20220323
Adding tb-nightly 2.9.0a20220323 to easy-inst

# 2. This section is for creating some important folders (IMPORTANT : Please Read)



*   First Run this cell

*   Once you've run the cell , you can see that a folder named "videos" will be created inside the Real-ESRGAN folder


*   Just drag and drop any number of videos from your computer and place them inside the "videos" folder. (videos you want to upscale)







In [ ]:
import os
from google.colab import files
import shutil

upload_folder = 'upload'
result_folder = 'results'
video_folder = 'videos'
video_result_folder = 'results_videos'
video_mp4_result_folder = 'results_mp4_videos'

if os.path.isdir(upload_folder):
  print(upload_folder+" exists")
else :
  os.mkdir(upload_folder)

if os.path.isdir(video_folder):
  print(video_folder+" exists")
else :
  os.mkdir(video_folder)

if os.path.isdir(video_result_folder):
  print(video_result_folder+" exists")
else :
  os.mkdir(video_result_folder)

if os.path.isdir(video_mp4_result_folder):
  print(video_mp4_result_folder+" exists")
else :
  os.mkdir(video_mp4_result_folder)

if os.path.isdir(result_folder):
  print(result_folder+" exists")
  
else :
  os.mkdir(result_folder)  




# 3. Inference (Please READ)



*   Then run this cell , it will take a good amount of time depending on the length of your video and how many videos you want to upscale

*   Once the upscaling process is complete , you can find your upscaled video results , under the "results_mp4_videos" folder.



*   By default the video is upscaled to a 25fps format,  in case you want to change that , it can be changed within the cell below , 
change the "fps" variable. (Can be found  towards the end part of the code section.)








In [ ]:
import cv2
import numpy as np
import glob
from os.path import isfile, join
import subprocess
from IPython.display import clear_output  

# assign directory
directory = '/content/GFPGAN/videos' #PATH_WITH_INPUT_VIDEOS
zee = 0

#deletes frames from previous video
for f in os.listdir(upload_folder): 
    os.remove(os.path.join(upload_folder, f))

#deletes upscaled frames from previous video
#for f in os.listdir('results/restored_imgs'): 
#    os.remove(os.path.join('results/restored_imgs', f))

#clearing previous .avi files
for f in os.listdir(video_result_folder): 
    os.remove(os.path.join(video_result_folder, f))
     
#clearing .mp4 result files
for f in os.listdir(video_mp4_result_folder): 
    os.remove(os.path.join(video_mp4_result_folder, f))


def convert_frames_to_video(pathIn,pathOut,fps):
    frame_array = []
    files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
    #for sorting the file names properly
    files.sort(key = lambda x: int(x[5:-4]))
    size2 = (0,0)

    for i in range(len(files)):
        filename=pathIn + files[i]
        #reading each files
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        size2 = size
        print(filename)
        #inserting the frames into an image array
        frame_array.append(img)
    out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size2)
    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
    out.release()
    
        
for filename in os.listdir(directory):
    
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
      
      
      print("PROCESSING :"+str(f)+"\n")
      # Read the video from specified path
    
      #video to frames
      cam = cv2.VideoCapture(str(f))

      try:

          # PATH TO STORE VIDEO FRAMES
          if not os.path.exists('/content/GFPGAN/upload'):
              os.makedirs('/content/GFPGAN/upload')

      # if not created then raise error
      except OSError:
          print ('Error: Creating directory of data')

      # frame
      currentframe = 0

      #clear all folders


      #deletes upscaled frames from previous video
      #for f in os.listdir(result_folder):
      #  os.remove(os.path.join(result_folder, f))      
      



      while(True):

          # reading from frame
          ret,frame = cam.read()

          if ret:
              # if video is still left continue creating images
              name = '/content/GFPGAN/upload/frame' + str(currentframe) + '.jpg'

              # writing the extracted images
              cv2.imwrite(name, frame)


                # increasing counter so that it will
                # show how many frames are created
              currentframe += 1
              print(currentframe)
          else:
              #deletes all the videos you uploaded for upscaling
              #for f in os.listdir(video_folder):
              #  os.remove(os.path.join(video_folder, f)) 

              break
      
        # Release all space and windows once done
      cam.release()
      cv2.destroyAllWindows()
      
      #apply super-resolution on all frames of a video
      #scale factor is by 3.5x  

      #!python inference_realesrgan.py -n RealESRGAN_x4plus -i upload --outscale 2 --half --face_enhance --tile 128
      !python inference_gfpgan.py -i upload -o results -v 1.2 -s 2 --bg_upsampler realesrgan

      #after upscaling just delete the source frames
      for f in os.listdir(upload_folder): 
          os.remove(os.path.join(upload_folder, f))     

      '''
      #rename all frames in "results" to remove the 'out' substring from the processing results
      paths = (os.path.join(root, filename)
              for root, _, filenames in os.walk('/content/GFPGAN/results')
              for filename in filenames)
      for path in paths:
          newname = path.replace('_out', '')
          if newname != path:
              os.rename(path, newname)      
      '''

      #convert super res frames to .avi  
      pathIn = '/content/GFPGAN/results/restored_imgs/'

      zee = zee+1
      fName = "video"+str(zee)  
      filenameVid = f"{fName}.avi"
        
      pathOut = "/content/GFPGAN/results_videos/"+filenameVid

      fps = 30.0 #change this to FPS of your source video

      convert_frames_to_video(pathIn, pathOut, fps)      

      #after processing frames converted to .avi video , delete upscaled frames from previous video
      for f in os.listdir('results/restored_imgs'): 
          os.remove(os.path.join('results/restored_imgs', f))

      #convert .avi to .mp4
      src = '/content/GFPGAN/results_videos/'
      dst = '/content/GFPGAN/results_mp4_videos/'

      for root, dirs, filenames in os.walk(src, topdown=False):
          #print(filenames)
          for filename in filenames:
              print('[INFO] 1',filename)
              try:
                  _format = ''
                  if ".flv" in filename.lower():
                      _format=".flv"
                  if ".mp4" in filename.lower():
                      _format=".mp4"
                  if ".avi" in filename.lower():
                      _format=".avi"
                  if ".mov" in filename.lower():
                      _format=".mov"

                  inputfile = os.path.join(root, filename)
                  print('[INFO] 1',inputfile)
                  outputfile = os.path.join(dst, filename.lower().replace(_format, ".mp4"))
                  subprocess.call(['ffmpeg', '-i', inputfile, outputfile])  
              except:
                  print("An exception occurred")        
  
      clear_output(wait=True)

      #clearing previous .avi files
      for f in os.listdir(video_result_folder): 
          os.remove(os.path.join(video_result_folder, f))

      #deletes frames from previous video
      #for f in os.listdir(upload_folder):
      #  os.remove(os.path.join(upload_folder, f))

        

      # if it is out of memory, try to use the `--tile` option
# We upsample the image with the scale factor X3.5

# Arguments
# -n, --model_name: Model names
# -i, --input: input folder or image
# --outscale: Output scale, can be arbitrary scale factore. 

Streaming output truncated to the last 5000 lines.
/content/GFPGAN/results/restored_imgs/frame1381.jpg
/content/GFPGAN/results/restored_imgs/frame1382.jpg
/content/GFPGAN/results/restored_imgs/frame1383.jpg
/content/GFPGAN/results/restored_imgs/frame1384.jpg
/content/GFPGAN/results/restored_imgs/frame1385.jpg
/content/GFPGAN/results/restored_imgs/frame1386.jpg
/content/GFPGAN/results/restored_imgs/frame1387.jpg
/content/GFPGAN/results/restored_imgs/frame1388.jpg
/content/GFPGAN/results/restored_imgs/frame1389.jpg
/content/GFPGAN/results/restored_imgs/frame1390.jpg
/content/GFPGAN/results/restored_imgs/frame1391.jpg
/content/GFPGAN/results/restored_imgs/frame1392.jpg
/content/GFPGAN/results/restored_imgs/frame1393.jpg
/content/GFPGAN/results/restored_imgs/frame1394.jpg
/content/GFPGAN/results/restored_imgs/frame1395.jpg
/content/GFPGAN/results/restored_imgs/frame1396.jpg
/content/GFPGAN/results/restored_imgs/frame1397.jpg
/content/GFPGAN/results/restored_imgs/frame1398.jpg
/content/GFPG

# 4. Download Results



*   All your upscaled .mp4 files will be inside the results_mp4_videos folder (inside Real-ESRGAN folder)


*   You can right click on the needed file and download it from there.






# 5. Run this cell after the batch of videos have been upscaled 
# (WARNING : Deletes processed data)



*   IMPORTANT : Read the comments inside the code section , because this DELETES previous frames and videos


In [ ]:


#deletes frames from previous video
for f in os.listdir(upload_folder): 
    os.remove(os.path.join(upload_folder, f))

#deletes upscaled frames from previous video
for f in os.listdir('results/restored_imgs'): 
    os.remove(os.path.join('results/restored_imgs', f))

#deletes all the videos you uploaded for upscaling
for f in os.listdir(video_folder): 
    os.remove(os.path.join(video_folder, f))

#clearing previous .avi files
for f in os.listdir(video_result_folder): 
    os.remove(os.path.join(video_result_folder, f))
     
#clearing .mp4 result files
for f in os.listdir(video_mp4_result_folder): 
    os.remove(os.path.join(video_mp4_result_folder, f))